In [1]:
!pip install mlflow boto3 awscli optuna xgboost imbalanced-learn

   ---------------------------------------- 0.0/383.4 kB ? eta -:--:--
   --- ------------------------------------ 30.7/383.4 kB ? eta -:--:--
   ------ -------------------------------- 61.4/383.4 kB 656.4 kB/s eta 0:00:01
   ------- ------------------------------- 71.7/383.4 kB 653.6 kB/s eta 0:00:01
   -------------- ----------------------- 143.4/383.4 kB 774.0 kB/s eta 0:00:01
   ------------------ ------------------- 184.3/383.4 kB 857.5 kB/s eta 0:00:01
   ----------------------- -------------- 235.5/383.4 kB 901.1 kB/s eta 0:00:01
   --------------------------------- ---- 337.9/383.4 kB 999.0 kB/s eta 0:00:01
   ---------------------------------------  378.9/383.4 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- 383.4/383.4 kB 919.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 1.3 MB/s eta 0:01:38
   ---------------------------------------- 0.0/124.9 MB 1.3 MB/s et

ERROR: Exception:
Traceback (most recent call last):
  File "E:\mlflow-without-pipeline\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "E:\mlflow-without-pipeline\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "E:\mlflow-without-pipeline\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "E:\mlflow-without-pipeline\venv\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "E:\mlflow-without-pipeline\venv\Lib\http\client.py", line 466, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "E:\mlflow-without-pipeline\venv\Lib\socket.py", line 706, in readinto
    r


   ---------------------- ---------------- 71.5/124.9 MB 167.4 kB/s eta 0:05:20
   ---------------------- ---------------- 71.5/124.9 MB 167.4 kB/s eta 0:05:20
   ---------------------- ---------------- 71.5/124.9 MB 167.8 kB/s eta 0:05:19
   ---------------------- ---------------- 71.5/124.9 MB 168.2 kB/s eta 0:05:18
   ---------------------- ---------------- 71.5/124.9 MB 168.2 kB/s eta 0:05:18
   ---------------------- ---------------- 71.5/124.9 MB 168.2 kB/s eta 0:05:18
   ---------------------- ---------------- 71.5/124.9 MB 168.2 kB/s eta 0:05:18
   ---------------------- ---------------- 71.5/124.9 MB 168.4 kB/s eta 0:05:17
   ---------------------- ---------------- 71.6/124.9 MB 168.5 kB/s eta 0:05:17
   ---------------------- ---------------- 71.6/124.9 MB 168.5 kB/s eta 0:05:17
   ---------------------- ---------------- 71.6/124.9 MB 168.9 kB/s eta 0:05:16
   ---------------------- ---------------- 71.6/124.9 MB 168.9 kB/s eta 0:05:16
   ---------------------- -------------

In [2]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-54-252-23-89.ap-southeast-2.compute.amazonaws.com:5000")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

2025/01/23 19:17:07 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5 - ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://yt-chrome-plugin-bucket/568898543910160492', creation_time=1737641683152, experiment_id='568898543910160492', last_update_time=1737641683152, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [4]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

ModuleNotFoundError: No module named 'optuna'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
from xgboost import XGBClassifier
import optuna

# Step 1: Load the CSV file
# Use a raw string or double backslashes to handle Windows file paths correctly.
file_path = r'E:\yt-chrome-plugin\chrome-plugin\data\processed\redit_preprocessing.csv'
df = pd.read_csv(file_path)

# Step 2: Remove rows with NaN values in the 'clean_comment' column
# Ensures that only rows with valid comments are retained.
df = df.dropna(subset=['clean_comment'])

# Step 3: Map values in the 'category' column
# Maps -1 to 2 (e.g., for negative sentiment), 0 to 0 (neutral), and 1 to 1 (positive).
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 4: Remove rows with NaN in the 'category' column
# Removes any rows where the mapping failed (e.g., values outside the mapping range).
df = df.dropna(subset=['category'])

# Step 5: Ensure the 'category' column is an integer type
# Converts the category column to integer type for compatibility with models.
df['category'] = df['category'].astype(int)

# Step 6: Define parameters for TF-IDF vectorizer
ngram_range = (1, 3)  # Use trigram analysis
max_features = 10000  # Maximum number of features for the TF-IDF vectorizer

# Step 7: Split the dataset into training and testing sets
# Splits data into 80% training and 20% testing while maintaining class balance.
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], random_state=42, test_size=0.20
)

# Step 8: Transform text data using TF-IDF
# Converts text data into numerical format suitable for machine learning models.
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Step 9: Handle class imbalance using SMOTE
# SMOTE generates synthetic samples for minority classes to balance the dataset.
smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Function to log experiments with MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    """
    Logs model training and evaluation metrics to MLflow.
    """
    with mlflow.start_run():
        # Set tags for the experiment
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log the model name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train the model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report metrics
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the trained model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Function for XGBoost optimization using Optuna
def objective_xgboost(trial):
    """
    Defines the objective function for Optuna to optimize XGBoost hyperparameters.
    """
    # Suggest hyperparameters for optimization
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    # Train an XGBoost model with the suggested hyperparameters
    model = XGBClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42
    )
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)

    # Return accuracy as the metric to optimize
    return accuracy_score(y_test, y_pred)

def run_optuna_experiment():
    """
    Runs Optuna optimization for XGBoost and logs the best model to MLflow.
    """
    # Create an Optuna study to maximize accuracy
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best hyperparameters and train the final model
    best_params = study.best_params
    best_model = XGBClassifier(
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        max_depth=best_params['max_depth'],
        random_state=42
    )

    # Log the best model with MLflow
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost with Optuna optimization
run_optuna_experiment()
